In [1]:
import numpy as np 
import pandas as pd 
import os
import cv2

In [2]:
# sensor_data = pd.read_csv('data/mysenior_sensor_data_output.csv')
# sensor_attitude = sensor_data[:][['Time (UTCG)','q1', 'q2', 'q3', 'q4']]
# print(sensor_attitude.head())
# sensor_attitude.to_csv('data/sensor_attitude.csv', index=False)

In [3]:
from dataclasses import dataclass
@dataclass()
class Debris:
  name: str
  time: pd.Timestamp
  magnitude: float
  position: np.ndarray
  
def read_debris_data(debris_file):
  debris_data = pd.read_csv(debris_file,header=None)
  is_header = debris_data.iloc[:,0].str.startswith('Time (UTCG)')
  group_ids = is_header.cumsum()-1
  group_count = group_ids.max() + 1
  magnitudes = np.random.normal(4, 0.5, group_count)
  headers = debris_data[is_header].iloc[:, 2]
  names = headers.str.rsplit('-', n=1, expand=True)[0].str.replace(' ', '')
  debris_name_list = names.unique().tolist()
  debris = []
  for grp_id, sub_df in debris_data.groupby(group_ids):
    print(grp_id,names.iloc[grp_id])
    coords = sub_df.iloc[1:, 1:4].to_numpy(dtype=np.float64)
    time = pd.to_datetime(sub_df.iloc[1:, 0])
    group_debris = [Debris(
        name = names.iloc[grp_id],
        time = time.iloc[i],
        magnitude = magnitudes[grp_id],
        position = coords[i]
      )for i in range(len(time))
    ]
    debris.append(group_debris)
  return debris, debris_name_list

In [4]:
@dataclass()
class Sensor:
  time: pd.Timestamp
  q: np.ndarray
  position_a: np.ndarray
  position: np.ndarray

def read_sensor_data(sensor_file):
  sensor_data = pd.read_csv(sensor_file)
  sensor = []
  sensor_data['Time (UTCG)'] = pd.to_datetime(sensor_data['Time (UTCG)'])
  time_ls = sensor_data['Time (UTCG)'].tolist()
  for index, row in sensor_data.iterrows():
    sensor.append(Sensor(
      time = row[0],
      q = row[7:11].to_numpy(dtype=np.float64),
      position_a = row[1:4].to_numpy(dtype=np.float64),
      position = np.zeros(3)
    ))
  return sensor,time_ls

In [5]:
debris_uv = pd.read_csv('debris_data.csv')
# sensor_attitude = pd.read_csv('data/sensor_attitude.csv')
sensor, time_ls = read_sensor_data('data/mysenior_sensor_data_output.csv')
debris, debris_name_list = read_debris_data('data/reduced_debris_data.csv')
time_list = debris_uv['time'].tolist()
debris_uv['time'] = pd.to_datetime(debris_uv['time'])

0 0_OPS_5721_09415
1 ASTRID_1_23464
2 COBE_20322
3 COSMOS_1176_11788
4 COSMOS_1181_11803
5 COSMOS_1302_12791
6 COSMOS_1357_13160
7 COSMOS_1431_13763
8 COSMOS_1476_14174
9 COSMOS_1637_15619
10 COSMOS_1930_18943
11 COSMOS_2008_19902
12 COSMOS_2270_23001
13 COSMOS_507_06120
14 COSMOS_617_06985
15 COSMOS_626_07005
16 COSMOS_645_07269
17 COSMOS_789_08591
18 DUMMY_MASS_1_24925
19 ESSA_9_03764
20 METEOR_1-15_06659
21 METEOR_1-23_08519
22 METEOR_PRIRODA_12585
23 SURCAL_159_02872
24 TIMATION_1_02847


In [6]:
class DebrisMatch:
  def __init__(self, name, index, u, v):
    self.name = name
    self.index = index
    self.u = u
    self.v = v

In [7]:
def debris_match(uv_dict, debris_data, sensor_q, sensor_pos_est):
  #参数：uv_dict:字典，键为编号，值为uv值  debris_dict:字典，键为编号，值为debris名字str  debris_data:所有debris在此刻的对象，长度为debris的数目  sensor_q:传感器姿态数据  sensor_pos_est:传感器位置数据，预估，有误差
  #返回：debris_dict:字典，键为编号，值为debris名字str；error,当匹配失败时，返回一个误差
  f = 0.01413
  dh = 0.000006
  dv = 0.000006
  H = 700
  debris_dict = {}
  if sensor_q[3] < 0:
    sensor_q = -sensor_q
  q0, q1, q2, q3 = sensor_q
  Msi = np.array([[q3**2+q0**2-q1**2-q2**2, 2*q3*q2+2*q0*q1,     -2*q3*q1+2*q0*q2],
                    [-2*q3*q2+2*q0*q1,    q3**2-q0**2+q1**2-q2**2, 2*q3*q0+2*q1*q2],
                    [2*q3*q1+2*q0*q2,     -2*q3*q0+2*q1*q2,    q3**2-q0**2-q1**2+q2**2]])
  match_ls = [] #存放计算的可能的目标碎片，目前只有碎片名称但没有碎片编号
  for debris0 in debris_data:
    name = debris0.name
    position = debris0.position
    vec = sensor_pos_est - position
    distance = np.linalg.norm(vec)
    vec = vec / distance
    A = np.dot(Msi, vec)
    u = f*A[0]/(dh*A[2])+H
    v = f*A[1]/(dv*A[2])+H
    if u < -H or u > H*3 or v < -H or v > H*3:
      continue
    match0 = DebrisMatch(name, -1, v, u) # 这里uv交换一个位置，因为在图像中，x轴是v轴，y轴是u轴
    match_ls.append(match0)

  for index, uv_true in uv_dict.items():
    u,v = uv_true
    distance_ls = []
    for match0 in match_ls:
      distance_ls.append(np.linalg.norm(np.array([u,v])-np.array([match0.u, match0.v])))
    min_distance = min(distance_ls)
    match_ls[distance_ls.index(min_distance)].index = index
  for match0 in match_ls:
    if match0.index == -1:
      continue
    debris_dict[match0.index] = match0.name
  
  return debris_dict

In [8]:
uv_dict_ls = []
for time0 in range(len(time_list)):
  uv_dict = {}
  for index, row in debris_uv.iterrows():
    if row[0] == time_ls[time0]:
      index = int(row[3])
      uv_dict[index] = [row[1], row[2]]
  sensor_q = sensor[time0].q
  sensor_pos_est = sensor[time0].position_a + np.random.normal(0, 1, 3) # 加噪声
  debris_data = []
  for i in range(len(debris)):
    debris0 = debris[i][time0]
    debris_data.append(debris0)
  debris_dict = debris_match(uv_dict, debris_data, sensor_q, sensor_pos_est)
  uv_dict_ls.append(debris_dict)

In [9]:
print(uv_dict_ls)
print(debris_dict)

[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-15_06659'}, {35: 'METEOR_1-1

In [10]:
final_dict = {}
uv_dict = {}
for debris_dict in uv_dict_ls:
  for index, name in debris_dict.items():
    if index in final_dict:
      final_dict[index].append(name)
    else:
      final_dict[index] = [name]
for index, name in final_dict.items():
  unique_name = list(set(name))
  frequency=[]
  for i in unique_name:
    frequency.append(name.count(i))
    print(index, i, name.count(i))
  uv_dict[index] = unique_name[frequency.index(max(frequency))]
print(uv_dict)

35 METEOR_1-15_06659 595
{35: 'METEOR_1-15_06659'}


In [11]:
print(uv_dict)

{35: 'METEOR_1-15_06659'}


In [12]:
columns=['time','name','target_index','u','v','x','y','z']
visible_debris = []
for index, row in debris_uv.iterrows():
  if row[3] in uv_dict:
    name = uv_dict[row[3]]
    time = row[0]
    target_index = row[3]
    x,y,z = debris[debris_name_list.index(name)][time_ls.index(time)].position
    u,v = row[1],row[2]
    visible_debris.append({'time':time,'name':name,'target_index':target_index,'u':u,'v':v,'x':x,'y':y,'z':z})
visible_debris = pd.DataFrame(visible_debris, columns=columns)
visible_debris.to_csv('visible_debris.csv',index=False)
print(visible_debris)

                       time               name  target_index           u  \
0   2023-04-25 16:00:05.000  METEOR_1-15_06659            35  884.714286   
1   2023-04-25 16:00:05.100  METEOR_1-15_06659            35  884.875000   
2   2023-04-25 16:00:05.200  METEOR_1-15_06659            35  885.000000   
3   2023-04-25 16:00:05.300  METEOR_1-15_06659            35  885.000000   
4   2023-04-25 16:00:05.400  METEOR_1-15_06659            35  885.000000   
..                      ...                ...           ...         ...   
640 2023-04-25 16:01:09.000  METEOR_1-15_06659            35  955.125000   
641 2023-04-25 16:01:09.100  METEOR_1-15_06659            35  955.285714   
642 2023-04-25 16:01:09.200  METEOR_1-15_06659            35  955.500000   
643 2023-04-25 16:01:09.300  METEOR_1-15_06659            35  955.875000   
644 2023-04-25 16:01:09.400  METEOR_1-15_06659            35  955.714286   

               v            x            y            z  
0     884.000000 -2776.315316

In [13]:
## test 
sensor_pos = sensor[50].position_a
u, v, x, y, z =visible_debris.iloc[1][3:8]
vec = np.array([x,y,z]) - sensor_pos
vec = vec / np.linalg.norm(vec)
q = sensor[50].q
q0, q1, q2, q3 = q
Msi = np.array([[q3**2+q0**2-q1**2-q2**2, 2*q3*q2+2*q0*q1,     -2*q3*q1+2*q0*q2],
                [-2*q3*q2+2*q0*q1,    q3**2-q0**2+q1**2-q2**2, 2*q3*q0+2*q1*q2],
                [2*q3*q1+2*q0*q2,     -2*q3*q0+2*q1*q2,    q3**2-q0**2-q1**2+q2**2]])
A = np.dot(Msi, vec)
print(A)
f = 0.01413
dh = 0.000006
dv = 0.000006
H = 700
vec0 = np.array([(H-v)*dv,(H-u)*dh,f])
print(np.linalg.norm(vec0),vec0/np.linalg.norm(vec0))

[0.07777046 0.07793326 0.99392015]
0.014216812973553532 [-0.07802382 -0.07802382  0.99389364]
